<a href="https://colab.research.google.com/github/MrFzovpec/mettre-marketing/blob/master/marketing_analysis/instagram/Instagramm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install pytorch-lightning

# Organizing the data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/MrFzovpec/mettre-marketing/master/marketing_analysis/instagram/instagram.csv')

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 
                 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1'])

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df['index_account'] = None

The accounts are depersonalized, so we need to indexate them

In [ ]:
index_acc = 0

In [ ]:
for x in df.iterrows():
  if x[0] == 0:
    df['index_account'][x[0]] = index_acc
    previous = x[1]['account_description']
    continue
  if x[1]['account_description'] == previous:
    df['index_account'][x[0]] = index_acc
  else:
    index_acc += 1
    df['index_account'][x[0]] = index_acc

  previous = x[1]['account_description']

In [ ]:
df.head()

,total_posts,text,likes,date,subscribers,subscribed,image_urls,account_description,index_account
0,155,here’s my fire/water girl oc w butterfly sleev...,"42,103",2020-06-11T20:32:23.000Z,"286,674",397,https://instagram.fbru2-1.fna.fbcdn.net/v/t51....,🎨 repost with credit!\n🍉 clip studio paint\n🍡 ...,0
1,155,here’s a tablet vs phone challenge bc i haven’...,"76,906",2020-06-09T20:35:07.000Z,"286,674",397,https://instagram.fbru2-1.fna.fbcdn.net/v/t51....,🎨 repost with credit!\n🍉 clip studio paint\n🍡 ...,0
2,155,here’s a sailor moon mermaid! ✨🌊🌊 i had fun dr...,"42,219",2020-06-05T20:50:10.000Z,"286,674",397,https://instagram.fbru2-1.fna.fbcdn.net/v/t51....,🎨 repost with credit!\n🍉 clip studio paint\n🍡 ...,0
3,155,here’s a support post for black artists and cr...,"81,812",2020-06-03T19:20:01.000Z,"286,674",397,https://instagram.fbru2-1.fna.fbcdn.net/v/t51....,🎨 repost with credit!\n🍉 clip studio paint\n🍡 ...,0
4,155,a milk carton vending machine! decorate the co...,"37,869",2020-05-28T20:37:42.000Z,"286,674",397,https://instagram.fbru2-1.fna.fbcdn.net/v/t51....,🎨 repost with credit!\n🍉 clip studio paint\n🍡 ...,0


# Creating a dataloader

This is going to be an LSTM model and it's going to work with the five previous examples. So, for example I wanna predict likes countable for some particular post, then I'm going to take 4 previous posts and make a prediction basing on their data. <br> <br>
Here I'll create a few classes which would provide that functionality

In [ ]:
class DatasetSamples():
  ''' This class will return a dataset samples in the format of 5 posts 
  (more or less). It's going to be kind of a sliding window'''
  def __init__(self, df, window_size=5):
    self.df = df
    self.window_size = window_size

  def get_window_of_posts(self):
    users = df['index_account'].unique()

    for user in users:
      user = int(user)
      user_df = df[df['index_account'] == user]
      user_df_len = len(user_df)
      user_posts_array = []

      # Identifying the indexes we're going to use to parse
      starts_index = 0
      final_index = user_df_len - self.window_size

      for index in range(starts_index, final_index):
        # Sometimes account doesn't have even 5 posts
        if len(user_df.iloc[index: index + self.window_size]) == 0:
          continue

        yield user_df.iloc[index: index + self.window_size]

The class above just samples a data with some particular window size and returns it in the format of generator of array of the posts (pandas df)

The following class is encoding the text and creating a tensor out of it

In [ ]:
from transformers import DistilBertTokenizer

In [ ]:
import torch.nn.functional as F

In [200]:
import torch

In [ ]:
class TextEncoder():
  def __init__(self, tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')):
    self.tokenizer = tokenizer

  def encode(self, samples):
    text_array = [] # creating a text array for keeping all the texts
    for i, sample in samples.iteritems():
      text_tensor = torch.tensor(self.tokenizer.encode(sample))
      text_array.append(text_tensor)

    return self.pad_and_stack(text_array)
  
  @staticmethod
  def get_largest_elem(array, dim=0):
    ''' The function identyfies the largest tensor over particular dimension '''
    max_len = 0
    for elem in array:
      # Runs over thought the array to identify the largest one
      if elem.shape[dim] > max_len:
        max_len = elem.shape[dim]

    return max_len

  def pad_and_stack(self, array, dim=0):
    ''' Function pads and stacks array over a new axis '''

    if dim == 0:
      largest = self.get_largest_elem(array) # gets the largest to pad
      array_for_stack = []
      for elem in array:
        # Pad the elements to get equal shapes
        elem = F.pad(elem, [0, largest - elem.shape[dim]])
        array_for_stack.append(elem)

    return torch.stack(array_for_stack)
      

The following class is the class which's going to encode images out of the link

In [ ]:
from PIL import Image

In [ ]:
import requests

In [ ]:
from io import BytesIO

In [ ]:
from torchvision import transforms

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class ImageEncoder():
  def __init__(self, size_index=2, transform=transforms.ToTensor()):
    self.size_index = size_index
    self.transform = transform

  def encode(self, samples):
    image_array = [] # creating an array for keeping all the images
    for i, sample in samples.iteritems():
      # Getting a clear link of an image
      link_href = sample.split(',')[self.size_index][:-5]
      image = self.get_img_from_remote_server(link_href)
      image = self.transform(image)
      image_array.append(image)
    
    return torch.stack(image_array)

  @staticmethod
  def get_img_from_remote_server(url):
    ''' Function gets an image from a remote server '''
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

This class will pack up a metadata into the tensor

In [ ]:
class MetaDataEncoder():
  def encode(self, samples):
    meta_array = [] # creating an array for keeping all the metadata
    for i, sample in samples.iteritems():
      meta_array.append(torch.tensor([int(sample.replace(',',''))]))

    return torch.stack(meta_array)

The following class is going to encode the date

In [ ]:
from datetime import datetime

In [ ]:
class DateEncoder():
  def encode(self, samples):
    date_array = []

    for i, sample in samples.iteritems():
      date = datetime.strptime(sample, '%Y-%m-%dT%H:%M:%S.%fZ')
      date = date.timestamp()
      date_array.append(torch.tensor([date]))

    return torch.stack(date_array)

The following class is going to manage the data and give the final one

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class DatasetManager(Dataset):
  def __init__(self, df, generator=DatasetSamples, text_encoder=TextEncoder(),
               image_encoder=ImageEncoder(), meta_data_encoder=MetaDataEncoder(),
               date_encoder=DateEncoder()):
    super().__init__()
    self.generator = generator(df)
    self.generator = self.generator.get_window_of_posts()
    self.data = [sample for sample in self.generator]

    # Encoders for different data
    self.text_encoder, self.account_description_encoder = text_encoder, text_encoder
    self.image_encoder = image_encoder
    self.date_encoder = date_encoder
    self.likes_encoder, self.comments_encoder = meta_data_encoder, meta_data_encoder
    self.total_posts_encoder, self.subscribers_encoder = meta_data_encoder, meta_data_encoder
    self.subscribed_encoder = meta_data_encoder

  def __getitem__(self, index):
    sample_data = self.data[index]

    return {
        'total_posts': self.total_posts_encoder.encode(sample_data['total_posts']),
        'text': self.text_encoder.encode(sample_data['text']),
        'likes': self.likes_encoder.encode(sample_data['likes']),
        'date': self.date_encoder.encode(sample_data['date']),
        'image': self.image_encoder.encode(sample_data['image_urls']),
        'subscribers': self.subscribers_encoder.encode(sample_data['subscribers']),
        'subscribed': self.subscribed_encoder.encode(sample_data['subscribed']),
        'account_description': self.account_description_encoder.encode(sample_data['account_description']),
    }

  def __len__(self):
    return len(self.data)

# Creating a model

The first version of the model is going to be an analysing LSTM model, which will analyse the last five posts and make the predictions about last one

At first we will create a model for the text analysis

In [177]:
from torch import nn

In [178]:
from transformers import DistilBertModel

In [179]:
class DistilBERTAnalysis(nn.Module):
  def __init__(self):
    super().__init__()
    self.transformer = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
    self.linear = nn.Linear(3072, 64)

  def forward(self, x):
    x = self.transformer(x)
    x = x[0][:, :4, :]
    x = x.view(x.shape[0], -1)
    x = self.linear(x)

    return x

The next model will be a model for images:

The first model for the image classification is going to be a VGG model as far as it gives really meaningfull vector on its top:

In [180]:
import torchvision.models as models

In [197]:
class InceptionV3Analysis(nn.Module):
  def __init__(self):
    super().__init__()
    self.imager = models.vgg19_bn(pretrained=False)
    self.imager.classifier = nn.Linear(25088, 64)

  def forward(self, x):
    return self.imager(x)

The following class is going to get a representational vector for the meta data:

In [198]:
class MetaLinearAnalyzator(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(4, 32)
    self.linear2 = nn.Linear(32, 64)

  def forward(self, x):
    x = self.linear1(x)
    x = F.leaky_relu(x)
    x = self.linear2(x)
    x = F.leaky_relu(x)

    return x

This class is going to recieve all the contexual vectors and make another contexual vector which is going to be passed to the following LSTM cell

In [199]:
class ContexualVectorCreator(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(256, 512)
  
  def forward(self, x):
    x = self.linear1(x)

    return x

It's an LSTM class, which going to finally give out a class prediction

In [ ]:
class LSTMPredictor(nn.Module):
  def __init__(self, memory_depth=512):
    self.memory_depth = memory_depth
    self.lstm_cell = nn.LSTMCell(512, self.memory_depth)

  def forward(self, x):
    ''' 
      !IMPORTANT: the function recieves an ARRAY OF TENSORS. NOT TORCH TENSOR
      Needs to write a part with a likes plussing 
    '''
    c_0 = self.get_first_cell_state(1)
    for i, vector in enumerate(x):
      h_0, c_0 = self.lstm_cell(vector, c_0)
      if len(x) - 1 == 

  def get_first_cell_state(self, samples_countable):
    return torch.zeros((samples_countable, self.memory_depth))

And here it is. This is the final model which is going to get predictions basing on the contexual vectors from the previous posts

In [ ]:
import pytorch_lightning as pl

In [ ]:
class EnsembledModelPredictor(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.text = DistilBERTAnalysis()
    self.images = InceptionV3Analysis()
    self.meta = MetaLinearAnalyzator()

  def forward(self, x):
    ''' Needs to rethink the architecture of the model '''
    text, acc_description = x['text'], x['account_description']

  def training_step(self):
    ''' So, here we will prepare and send the data to the post ''